In [ ]:
# Over the course of this tutorial I will refer to solar images as maps, or a set of them as mapsequence.
# Beware that the peek function even though provides a nice interactive way to view your maps but clogs your memory.
# As of now I still havent found a way to clear the figures from the memory. So the more you use the peek function the slower the code will get.
# You can avoid it by using using plot instead of peek but it will be lot less interactive .
# If the code gets very slow just restart the kernal start again with your new insights .

In [ ]:
%matplotlib qt

import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from glob import glob
from sunpy.map import Map
from datetime import datetime

import astropy.units as u
from astropy.coordinates import SkyCoord
from matplotlib.path import Path
from sunpy import timeseries as ts
from tqdm import *

from sunkit_image.coalignment import calculate_match_template_shift as cal_shift
from sunkit_image.coalignment import apply_shifts 

from matplotlib.lines import Line2D
from matplotlib.colors import PowerNorm
import csv
from sunpy import timeseries as ts

$\huge \text{Next up we have a couple of functions that will help us along the way.}$  

Python functions are named, reusable blocks of code designed to perform specific tasks.  

They promote modularity, code reuse, and easier maintenance in programming.  


In [ ]:
# This function exposure normalises your data (new data = old data/exposure ) and changes the header too.
# I am also setting all negative values to zero after normalisation.
def exp_norm(map_sequence):
    mod_rois = []
    for smap in tqdm(map_sequence):
        esf = (smap.meta['cmd_expt'])/1e3 #exposure scaling factor
        data = smap.data/esf   
        data[data<0] = 0
        smap.meta['cmd_expt'] = 1000 #changing the exposure to 1 sec
        mod_rois.append(Map(data,smap.meta))
    mod_rois = Map(mod_rois,sequence=True)
    return mod_rois

In [3]:
# This function will prompt you to select corners and will return the resulting quadilateral as a map.

def submap_with_ginput(sunpy_map):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection=sunpy_map)
    sunpy_map.plot(axes=ax)
    ax.set_title("Click two corners of the ROI (bottom-left and top-right)")

    # Wait for 2 clicks
    pts = plt.ginput(2, timeout=0)
    plt.close(fig)

    if len(pts) != 2:
        raise RuntimeError("ROI selection cancelled or failed.")

    (x1, y1), (x2, y2) = pts
    bottom_left = (min(x1, x2), min(y1, y2)) * u.pixel
    top_right = (max(x1, x2), max(y1, y2)) * u.pixel

    submap = sunpy_map.submap(bottom_left=bottom_left, top_right=top_right)
    return submap

In [4]:
# This function takes your map sequence as input and returns coaligned maps as output.
# Check Coalignment module for more info. 
# Vist *https://docs.sunpy.org/projects/sunkit-image/en/latest/code_ref/coalignment.html* to know more about the functions used.

def co_align(map_sequence,template,layer_index=0,clip=False):
    shifts = cal_shift(map_sequence,layer_index=layer_index,template=template)
    plate_scale = map_sequence[0].scale[0]
    if clip is False:
        coaligned_maps = apply_shifts(map_sequence, xshift=-shifts['x']/plate_scale, yshift=-shifts['y']/plate_scale, clip=False)
    if clip is True:
        coaligned_maps = apply_shifts(map_sequence, xshift=-shifts['x']/plate_scale, yshift=-shifts['y']/plate_scale, clip=True)
    return coaligned_maps

In [ ]:
# crops the maps in the mapsequence using the corners of the sample map provided.

def crop_maps(mc,sample):
    z=[]
    b_left =  sample.bottom_left_coord
    t_right = sample.top_right_coord
    x1,y1 = sample.world_to_pixel(b_left)
    x2,y2 = sample.world_to_pixel(t_right)
    for temp in mc:
        z.append(temp.submap(bottom_left = [x1,y1]*u.pix,top_right = [x2,y2]*u.pix))  
    z=Map(z,sequence=True)
    return z

In [6]:
# This function uses the co_align and submap_with_ginput function internally to align your mapsequence and plots the aligned maps. 

def align(mc,index=0):
    
    templ = submap_with_ginput(mc[index])
    fig=plt.figure()
    ax=fig.add_subplot(111,projection=templ)
    templ.plot(axes=ax)
    
    ax.set_title("Click anywhere to close the template and begin coalignment",fontweight="bold")
    plt.waitforbuttonpress()
    plt.close()
    
    aligned = co_align(map_sequence=mc,layer_index=0,template=templ,clip=False)
    aligned.peek();plt.show();plt.colorbar()
    return aligned

In [ ]:
# function to make lightcurve by adding all the pixels inside the mask for the entire mapsequence.

def make_lc(mask,map_sequence):
    lc=[];time=[]
    for smap in map_sequence:
        data = smap.data
        lc.append(np.sum(data[mask]))
        time.append(datetime.strptime(smap.meta['DHOBT_DT'].split('.')[0],"%Y-%m-%dT%H:%M:%S")) 
        #output = ['2024-11-10T13:59:58', '511920000'] to ignore the nanosec part you only take index 0
    return time,lc

$\huge \text{We start by loading data for different filters as sunpy maps}$

In [7]:
# Lets start with NB03
basepath = 'data/'

# loading all the NB03 files
files3 = sorted(glob(basepath+'nb03/*NB03.fits'))

# exposure normailizing the  
smaps3 = exp_norm(Map(files3,sequence=True))

#sometimes the data might not be of the same size. Hence we crop the remaining data to the size of the smaps3[0]
smaps3 = crop_maps(smaps3,smaps3[0])  

100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 51/51 [00:01<00:00, 28.28it/s]


In [ ]:
# Loading rest of the dataset
files1 = sorted(glob(basepath+'nb01/*NB01.fits'))
smaps1 = crop_maps(exp_norm(Map(files1,sequence=True)),smaps3[0])

files2 = sorted(glob(basepath+'nb02/*NB02.fits'))
smaps2 = crop_maps(exp_norm(Map(files2,sequence=True)),smaps3[0])

files4 = sorted(glob(basepath+'nb04/*NB04.fits'))
smaps4 = crop_maps(exp_norm(Map(files4,sequence=True)),smaps3[0])

files5 = sorted(glob(basepath+'nb05/*NB05.fits'))
smaps5 = crop_maps(exp_norm(Map(files5,sequence=True)),smaps3[0])

files6 = sorted(glob(basepath+'nb06/*NB06.fits'))
smaps6 = crop_maps(exp_norm(Map(files6,sequence=True)),smaps3[0])

files7 = sorted(glob(basepath+'nb07/*NB07.fits'))
smaps7 = crop_maps(exp_norm(Map(files7,sequence=True)),smaps3[0])

files8 = sorted(glob(basepath+'nb08/*NB08.fits'))
smaps8 = crop_maps(exp_norm(Map(files8,sequence=True)),smaps3[0])

In [ ]:
# Have a look at any map u like.
smaps6.peek()

$\huge \text{Align first image of each filter with each other.}$

In [ ]:
# start by making a template. I prefer using NB03 to make a template.
# For more tips on coalignment, look up the coalignment module.

template = submap_with_ginput(smaps3[0])
template.plot()

In [ ]:
# make a map sequence of first images of all the different filters.

temp = Map(smaps1[0],smaps2[0],smaps3[0],smaps4[0],smaps5[0],smaps6[0],smaps7[0],smaps8[0],sequence=True)

filter_index = {}
for i,tmap in enumerate(temp):
    filter_index[f"{tmap.meta['ftr_name']}"] = i
filter_index

In [ ]:
# align the images to the NB03 image using the template
index = 5 # we are using 5 because it corresponds to NB03 map in the map sequence. You can choose others also.It just aligns everything to that particular frame
temp = co_align(temp,layer_index=index,template=template)
temp.peek();plt.show()

In [ ]:
# if the coalignment does not work in the above cell, use this template to coalign

# bottom_left = SkyCoord(-407*u.arcsec, 397*u.arcsec, frame=smaps3[0].coordinate_frame)
# top_right = SkyCoord(-348*u.arcsec, 442*u.arcsec, frame=smaps3[0].coordinate_frame)
# template1 = smaps3[0].submap(bottom_left,top_right=top_right)
# template1.peek()

In [ ]:
# replace the first map of the different filters with the one aligned with NB03

smaps1.maps[0]  = temp[filter_index[f"{smaps1[1].meta['ftr_name']}"]]
smaps2.maps[0]  = temp[filter_index[f"{smaps2[1].meta['ftr_name']}"]]
smaps3.maps[0]  = temp[filter_index[f"{smaps3[1].meta['ftr_name']}"]]
smaps4.maps[0]  = temp[filter_index[f"{smaps4[1].meta['ftr_name']}"]]
smaps5.maps[0]  = temp[filter_index[f"{smaps5[1].meta['ftr_name']}"]]
smaps6.maps[0]  = temp[filter_index[f"{smaps6[1].meta['ftr_name']}"]]
smaps7.maps[0]  = temp[filter_index[f"{smaps7[1].meta['ftr_name']}"]]
smaps8.maps[0]  = temp[filter_index[f"{smaps8[1].meta['ftr_name']}"]]

$\huge \text{  Now we align all the images of a particular filter to the first image.}$
$\huge \text{  "align" function will make a template for you and then align all images to the first image using that template.}$ 
$\huge \text{  This might take a few tries.}$

$\huge \text{  Keep trying until you are confident with the coalignment. Have a look at the coalignment module for tips on selecting a template.}$


In [ ]:
# The align function will ask you to select bottom left and top right corner to select a template.
# It then shows you the template and waits for you to click and then starts coalignment.
# After the data is coaligned, it gets plotted for you to have a look.

coaligned_maps1 = align(smaps1)

In [31]:
coaligned_maps2 = align(smaps2)

In [33]:
coaligned_maps3 = align(smaps3)

In [ ]:
coaligned_maps4 = align(smaps4)

In [ ]:
coaligned_maps5 = align(smaps5)

In [ ]:
coaligned_maps6 = align(smaps6)

In [ ]:
coaligned_maps7 = align(smaps7)

In [ ]:
coaligned_maps8 = align(smaps8)

In [ ]:
# Here we are taking a 60% contour during the peak of the flare in NB03 filter. In this case the peak is around 24th NB03 frame. 
# Feel free to choose any other frame as well but it will change the ligtcurves that you will get. 

j =24
plt.figure()
contour_level = np.max(coaligned_maps3[j].data)*.6
contours = plt.contour(coaligned_maps3[j].data,levels=[contour_level],colors='red')
plt.imshow(coaligned_maps3[j].data,origin='lower')
plt.show()

In [ ]:
# creating a mask for the contour
path = contours.get_paths()[0]
data = coaligned_maps3[j].data
mask = np.zeros(data.shape, dtype=bool)
x, y = np.meshgrid(np.arange(data.shape[1]), np.arange(data.shape[0]))
points = np.vstack((x.flatten(), y.flatten())).T 
mask = path.contains_points(points).reshape(data.shape)
plt.figure()
plt.imshow(mask,origin='lower')
plt.show()

In [ ]:
# This is to make contours on your maps and then save them as png. Choose power norm paramters according to how your image looks.

j=24 #flare peak in NB03 for this flare

contour_level1 = np.max(coaligned_maps3[j].data)*.3 
contour_level2 = np.max(coaligned_maps3[j].data)*.6

for i,smap in enumerate(coaligned_maps3):
    fig = plt.figure()
    ax = fig.add_subplot(projection=coaligned_maps3[0])
    ax.imshow(smap.data,cmap='suit_nb03',norm=PowerNorm(0.4,vmin=0,vmax=42000), origin='lower') #plotting just the image data
    
    # smap.plot(axes=ax,norm=PowerNorm(0.4,vmin=0000,vmax=42000), origin='lower')  #this one plots the map with WCS not just data
    # plt.colorbar() 
    
    c1 = plt.contour(coaligned_maps3[j].data,levels=[contour_level1],colors='red',linewidth=0.5,label='30%')  #making 30 percent contour from flare peak
    c2 = plt.contour(coaligned_maps3[j].data,levels=[contour_level2],colors='cyan',linewidth=0.5,label='60%') #making 60 percent contour from flare peak
    legend_elements = [Line2D([0], [0], color='red',  lw=2, label = '30% Contour'),
                       Line2D([0], [0], color='cyan', lw=2, label = '60% Contour')
                      ]
    ax.legend(handles=legend_elements, loc='upper left',bbox_to_anchor=(.9, 1.16),frameon=False)
   
    ax.grid(False)
    # break
    plt.savefig(f'images/{i:03d}.png',dpi=300)
    plt.close()

In [ ]:
time1,lc1 = make_lc(mask,coaligned_maps1)
time2,lc2 = make_lc(mask,coaligned_maps2)
time3,lc3 = make_lc(mask,coaligned_maps3)
time4,lc4 = make_lc(mask,coaligned_maps4)
time5,lc5 = make_lc(mask,coaligned_maps5)
time6,lc6 = make_lc(mask,coaligned_maps6)
time7,lc7 = make_lc(mask,coaligned_maps7)
time8,lc8 = make_lc(mask,coaligned_maps8)

In [ ]:
# loading the GOES file provided along with the dataset. This file should be in the same directory as this notebook or you can change the path accordingly.
file_goes = glob('*.nc')
goes_18 = ts.TimeSeries(file_goes)
xrs_short = goes_18.data['xrsb']
goes_18.peek()

In [ ]:
fig,ax = plt.subplots(3,1,figsize=(13,11),sharex=True)
ax = ax.flatten()

ax[2].plot(time1,lc1/max(lc1),'.-', label='NB01')
ax[1].plot(time2,lc2/max(lc2),'m.-',label='NB02')
ax[0].plot(time3,lc3/max(lc3),'.-.',label='NB03')
ax[0].plot(time4,lc4/max(lc4),'yh-',label='NB04')
ax[1].plot(time5,lc5/max(lc5),'c^-',label='NB05')
ax[2].plot(time6,lc6/max(lc6),'s-', label='NB06')
ax[2].plot(time7,lc7/max(lc7),'.-', label='NB07')
ax[0].plot(time8,lc8/max(lc8),'h--',label='NB08')
ax[0].set_xlim(time3[0],time3[-1])

for i in range(3):
    ax[i].legend(frameon=False)
    ax[i].axvline(x=time3[np.argmax(lc3)],color='black',linestyle='dotted')
    ax[i].axvline(x=xrs_short.idxmax(),color='green',linestyle='dashed')
    ax1 = ax[i].twinx()
    ax1.plot(xrs_short,'r',label='GOES',)
    ax1.set_ylim(1e-6,9e-4)
    ax1.legend(frameon=False,loc=2)
    ax1.set_yscale('log')

fig.subplots_adjust(hspace=0.0)
fig.text(0.02, 0.5, 'SUIT Normalized Counts', va='center', rotation='vertical', fontsize=16)
fig.text(0.96, 0.5, 'Flare Class W/m$^{2}$',  va='center', rotation=270, fontsize=16)
fig.subplots_adjust(right=0.9) 
plt.suptitle('60% Contour LightCurves',fontsize=18,y=0.92)
fig.supxlabel('TIME(UT)',fontsize=16,y=0.04)

# Create proxy handles for vertical lines
vline_black = Line2D([0], [0], color='black', linestyle='dotted', label='NB03 Peak')
vline_red = Line2D([0], [0], color='green', linestyle='dashed', label='GOES Peak ')
# Add global legend outside the plot (right side)
fig.legend(handles=[vline_black, vline_red], loc='center right', frameon=False, fontsize=12, bbox_to_anchor=(.27, .9))
plt.show()
# plt.savefig("lightcurves60%.pdf",dpi=400)


In [ ]:
# saving the lightcurve you just produced. We are using zip_longest because sometime the number of frames in different filters might not be the same. 
from itertools import zip_longest
with open('nblc_60contour.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['time1','lc1','time2','lc2','time3','lc3','time4','lc4','time5','lc5','time6','lc6','time7','lc7','time8','lc8'])   # Header  
    writer.writerows(zip_longest(time1, lc1, time2, lc2, time3, lc3, time4, lc4,time5, lc5, time6, lc6, time7, lc7, time8, lc8, fillvalue='')) # data